In [371]:
import pandas as pd
import anndata as ad
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix, csc_matrix
from scipy.io import mmread, mmwrite
import os
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.utils.multiclass import type_of_target

In [372]:
###Seurat

In [373]:
##读取

In [ ]:
train_id <- "Dataset35"
test_id <- "Dataset36"

In [521]:
DORC=pd.read_csv("../results/DORC/"+ test_id ".csv",index_col=0,sep=' ')

In [523]:
true=pd.read_csv("../results/Seurat/Seurat_" + train_id + "_" + test_id + "_true.csv",sep=",",index_col=0)
pred=pd.read_csv("../results/Seurat/Seurat_" + train_id + "_" + test_id + "_pred.csv",sep=",",index_col=0)

In [525]:
pred=pred.sort_index()
true=true.sort_index()
pred=pred.T.sort_index()
true=true.T.sort_index()

In [533]:
# true=true.T
pred=pred.T
pred=pred.loc[true.index.values.flatten(),]

In [537]:
peaks,index_y =np.unique(DORC.peak,return_inverse=True)
genes,index_x =np.unique(DORC.gene,return_inverse=True)
DORC_gene= coo_matrix((np.ones(shape=(len(index_x),)),(index_x,index_y)))

In [538]:
DORC_gene.index=genes
DORC_gene.columns=peaks

In [539]:
#dorc转数据框
DORC_gene_matrix=pd.DataFrame(DORC_gene.toarray(),index=DORC_gene.index,columns=DORC_gene.columns)

In [540]:
DORC_gene_matrix=DORC_gene_matrix[(DORC_gene_matrix.sum(axis=1)>5)]

In [541]:
DORC_gene_matrix

,chr1-100038644-100039418,chr1-1000558-1001151,chr1-10022422-10023370,chr1-10032403-10033291,chr1-100351075-100351920,chr1-100400115-100401032,chr1-100423659-100424535,chr1-100439878-100440768,chr1-100441357-100442289,chr1-100443760-100444664,...,chrX-79133628-79134549,chrX-79144122-79145005,chrX-79161413-79162332,chrX-79258820-79259745,chrX-79270101-79271043,chrX-79325001-79325941,chrX-79337122-79338031,chrX-79360959-79361907,chrX-79366750-79367634,chrX-79400362-79401306
A2M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ABHD17A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ABLIM1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ABR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ABTB2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZNF703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZNF710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZNF831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [542]:
true_1 = true.loc[DORC_gene_matrix.columns.values.flatten(),]

In [543]:
pred_1 = pred.loc[DORC_gene_matrix.columns.values.flatten(),]

In [544]:
#得到gene cell矩阵 
true_matrix=DORC_gene_matrix.dot(true_1)

In [545]:
pred_matrix=DORC_gene_matrix.dot(pred_1)

In [546]:
pccs_row=[]
for i in range(0,true_matrix.shape[0]):
    corr = np.corrcoef(true_matrix.iloc[i], pred_matrix.iloc[i])[0,1]
    pccs_row.append(corr)

In [547]:
pccs_col=[]
for i in range(0,true_matrix.shape[1]):
    corr = np.corrcoef(true_matrix.iloc[:,i], pred_matrix.iloc[:,i])[0,1]
    pccs_col.append(corr)

In [548]:
pccs_row_df=pd.DataFrame(data=pccs_row)
np.median(pccs_row_df)

0.29216965682741775

In [549]:
pccs_col_df=pd.DataFrame(data=pccs_col)
np.nanmedian(pccs_col_df)

0.5066349016370346

In [550]:
pccs_row_df.to_csv("../results/DORC/Seurat_" + train_id + "_" + test_id + "_dorc_pcc.csv")
pccs_col_df.to_csv("../results/DORC/Seurat_" + train_id + "_" + test_id + "_cell_pcc.csv")

In [551]:
def CMD_dist(A, B):
    x = np.trace(A.dot(B))
    y = np.linalg.norm(A,'fro')*np.linalg.norm(B,'fro')
    return 1- x/(y+1e-8)

In [552]:
corr_pred = np.corrcoef(pred_matrix)
corr_true = np.corrcoef(true_matrix)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
Seurat_cmd_dorc=CMD_dist(corr_pred,corr_true)

In [553]:
corr_pred = np.corrcoef(pred_matrix.T)
corr_true = np.corrcoef(true_matrix.T)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
Seurat_cmd_cell=CMD_dist(corr_pred,corr_true)

In [554]:
Seurat_cmd_dorc

0.04557614442637703

In [555]:
Seurat_cmd_cell

0.06322720323626996

In [556]:
##LIGER

In [557]:
true=pd.read_csv("../results/LIGER/LIGER_" + train_id + "_" + test_id + "_true.csv",sep=",",index_col=0)
pred=pd.read_csv("../results/LIGER/LIGER_" + train_id + "_" + test_id + "_pred.csv",sep=",",index_col=0)
true=true.T
pred=pred.sort_index()
true=true.sort_index()
pred=pred.T.sort_index()
true=true.T.sort_index()
true=true.loc[pred.index]
pred=pred.loc[:,true.columns]
pred=pred.T.sort_index()
true=true.T.sort_index()

In [559]:
# pred=pred.T

In [560]:
true=true.loc[pred.index]
pred=pred.loc[:,true.columns]

In [561]:
true_1 = true.loc[DORC_gene_matrix.columns.values.flatten(),]

In [562]:
pred_1 = pred.loc[DORC_gene_matrix.columns.values.flatten(),]

In [563]:
#得到gene cell矩阵 
true_matrix=DORC_gene_matrix.dot(true_1)

In [564]:
pred_matrix=DORC_gene_matrix.dot(pred_1)

In [565]:
pccs_row=[]
for i in range(0,true_matrix.shape[0]):
    corr = np.corrcoef(true_matrix.iloc[i], pred_matrix.iloc[i])[0,1]
    pccs_row.append(corr)

In [566]:
pccs_col=[]
for i in range(0,true_matrix.shape[1]):
    corr = np.corrcoef(true_matrix.iloc[:,i], pred_matrix.iloc[:,i])[0,1]
    pccs_col.append(corr)

In [567]:
pccs_row_df=pd.DataFrame(data=pccs_row)
np.median(pccs_row_df)

0.3936195296152186

In [568]:
pccs_col_df=pd.DataFrame(data=pccs_col)
np.nanmedian(pccs_col_df)

0.5938711696190208

In [569]:
pccs_row_df.to_csv("../results/DORC/LIGER_" + train_id + "_" + test_id + "_dorc_pcc.csv")
pccs_col_df.to_csv("../results/DORC/LIGER_" + train_id + "_" + test_id + "_cell_pcc.csv")

In [570]:
def CMD_dist(A, B):
    x = np.trace(A.dot(B))
    y = np.linalg.norm(A,'fro')*np.linalg.norm(B,'fro')
    return 1- x/(y+1e-8)

In [571]:
corr_pred = np.corrcoef(pred_matrix)
corr_true = np.corrcoef(true_matrix)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
LIGER_cmd_dorc=CMD_dist(corr_pred,corr_true)

In [572]:
corr_pred = np.corrcoef(pred_matrix.T)
corr_true = np.corrcoef(true_matrix.T)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
LIGER_cmd_cell=CMD_dist(corr_pred,corr_true)

In [573]:
LIGER_cmd_dorc

0.19214626650532107

In [574]:
LIGER_cmd_cell

0.06006700026160883

In [575]:
# CMAE
true=pd.read_csv("../results/CMAE/CMAE_" + train_id + "_" + test_id + "_true.csv",sep=",",index_col=0)
pred=pd.read_csv("../results/CMAE/CMAE_" + train_id + "_" + test_id + "_pred.csv",sep=",",index_col=0)true=true.T
pred.index=true.index
pred.columns=true.columns
pred=pred.sort_index()
true=true.sort_index()
pred=pred.T.sort_index()
true=true.T.sort_index()
pred=pred.T.sort_index()
true=true.T.sort_index()

In [577]:
true_1 = true.loc[DORC_gene_matrix.columns.values.flatten(),]

In [578]:
pred_1 = pred.loc[DORC_gene_matrix.columns.values.flatten(),]

In [579]:
#得到gene cell矩阵 
true_matrix=DORC_gene_matrix.dot(true_1)

In [580]:
pred_matrix=DORC_gene_matrix.dot(pred_1)

In [581]:
pccs_row=[]
for i in range(0,true_matrix.shape[0]):
    corr = np.corrcoef(true_matrix.iloc[i], pred_matrix.iloc[i])[0,1]
    pccs_row.append(corr)

In [582]:
pccs_col=[]
for i in range(0,true_matrix.shape[1]):
    corr = np.corrcoef(true_matrix.iloc[:,i], pred_matrix.iloc[:,i])[0,1]
    pccs_col.append(corr)

In [583]:
pccs_row_df=pd.DataFrame(data=pccs_row)
np.median(pccs_row_df)

-0.008000488604740173

In [584]:
pccs_col_df=pd.DataFrame(data=pccs_col)
np.nanmedian(pccs_col_df)

0.24922789252022054

In [585]:
pccs_row_df.to_csv("../results/DORC/CMAE_" + train_id + "_" + test_id + "_dorc_pcc.csv")
pccs_col_df.to_csv("../results/DORC/CMAE_" + train_id + "_" + test_id + "_cell_pcc.csv")

In [586]:
def CMD_dist(A, B):
    x = np.trace(A.dot(B))
    y = np.linalg.norm(A,'fro')*np.linalg.norm(B,'fro')
    return 1- x/(y+1e-8)

In [587]:
corr_pred = np.corrcoef(pred_matrix)
corr_true = np.corrcoef(true_matrix)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
CMAE_cmd_dorc=CMD_dist(corr_pred,corr_true)

In [588]:
corr_pred = np.corrcoef(pred_matrix.T)
corr_true = np.corrcoef(true_matrix.T)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
CMAE_cmd_cell=CMD_dist(corr_pred,corr_true)

In [589]:
CMAE_cmd_dorc

0.5749763532679146

In [590]:
CMAE_cmd_cell

0.22210802881072844

In [591]:
#DANCE
true=pd.read_csv("../results/ScMoGCN/ScMoGCN_" + train_id + "_" + test_id + "_true.csv",sep=",",index_col=0)
pred=pd.read_csv("../results/ScMoGCN/ScMoGCN_" + train_id + "_" + test_id + "_pred.csv",sep=",",index_col=0)pred=pred.T
pred.index=true.index
pred.columns=true.columns
pred=pred.sort_index()
true=true.sort_index()
pred=pred.T.sort_index()
true=true.T.sort_index()

In [595]:
true_1 = true.loc[DORC_gene_matrix.columns.values.flatten(),]

In [596]:
pred_1 = pred.loc[DORC_gene_matrix.columns.values.flatten(),]

In [597]:
#得到gene cell矩阵 
true_matrix=DORC_gene_matrix.dot(true_1)

In [598]:
pred_matrix=DORC_gene_matrix.dot(pred_1)

In [599]:
pccs_row=[]
for i in range(0,true_matrix.shape[0]):
    corr = np.corrcoef(true_matrix.iloc[i], pred_matrix.iloc[i])[0,1]
    pccs_row.append(corr)

In [600]:
pccs_col=[]
for i in range(0,true_matrix.shape[1]):
    corr = np.corrcoef(true_matrix.iloc[:,i], pred_matrix.iloc[:,i])[0,1]
    pccs_col.append(corr)

In [601]:
pccs_row_df=pd.DataFrame(data=pccs_row)
np.median(pccs_row_df)

-0.014993073381835793

In [602]:
pccs_col_df=pd.DataFrame(data=pccs_col)
np.nanmedian(pccs_col_df)

-0.03325624136961446

In [603]:
pccs_row_df.to_csv("../results/DORC/ScMoGCN_" + train_id + "_" + test_id + "_dorc_pcc.csv")
pccs_col_df.to_csv("../results/DORC/ScMoGCN_" + train_id + "_" + test_id + "_cell_pcc.csv")

In [604]:
def CMD_dist(A, B):
    x = np.trace(A.dot(B))
    y = np.linalg.norm(A,'fro')*np.linalg.norm(B,'fro')
    return 1- x/(y+1e-8)

In [605]:
corr_pred = np.corrcoef(pred_matrix)
corr_true = np.corrcoef(true_matrix)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
DANCE_cmd_dorc=CMD_dist(corr_pred,corr_true)

In [606]:
corr_pred = np.corrcoef(pred_matrix.T)
corr_true = np.corrcoef(true_matrix.T)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
DANCE_cmd_cell=CMD_dist(corr_pred,corr_true)

In [607]:
DANCE_cmd_dorc

0.9887154493986612

In [608]:
DANCE_cmd_cell

0.2201628417811291

In [609]:
#KAUST
true=pd.read_csv("../results/LS_Lab/LS_Lab_" + train_id + "_" + test_id + "_true.csv",sep=",",index_col=0)
pred=pd.read_csv("../results/LS_Lab/LS_Lab_" + train_id + "_" + test_id + "_pred.csv",sep=",",index_col=0)pred.index=true.index
pred.columns=true.columns
pred=pred.sort_index()
true=true.sort_index()
pred=pred.T.sort_index()
true=true.T.sort_index()

In [613]:
true_1 = true.loc[DORC_gene_matrix.columns.values.flatten(),]

In [614]:
pred_1 = pred.loc[DORC_gene_matrix.columns.values.flatten(),]

In [615]:
#得到gene cell矩阵 
true_matrix=DORC_gene_matrix.dot(true_1)

In [616]:
pred_matrix=DORC_gene_matrix.dot(pred_1)

In [617]:
pccs_row=[]
for i in range(0,true_matrix.shape[0]):
    corr = np.corrcoef(true_matrix.iloc[i], pred_matrix.iloc[i])[0,1]
    pccs_row.append(corr)

In [618]:
pccs_col=[]
for i in range(0,true_matrix.shape[1]):
    corr = np.corrcoef(true_matrix.iloc[:,i], pred_matrix.iloc[:,i])[0,1]
    pccs_col.append(corr)

In [619]:
pccs_row_df=pd.DataFrame(data=pccs_row)
np.median(pccs_row_df)

0.3952808480175216

In [620]:
pccs_col_df=pd.DataFrame(data=pccs_col)
np.nanmedian(pccs_col_df)

0.5982380045398107

In [621]:
pccs_row_df.to_csv("../results/DORC/LS_Lab_" + train_id + "_" + test_id + "_dorc_pcc.csv")
pccs_col_df.to_csv("../results/DORC/LS_Lab_" + train_id + "_" + test_id + "_cell_pcc.csv")

In [622]:
def CMD_dist(A, B):
    x = np.trace(A.dot(B))
    y = np.linalg.norm(A,'fro')*np.linalg.norm(B,'fro')
    return 1- x/(y+1e-8)

In [623]:
corr_pred = np.corrcoef(pred_matrix)
corr_true = np.corrcoef(true_matrix)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
KAUST_cmd_dorc=CMD_dist(corr_pred,corr_true)

In [624]:
corr_pred = np.corrcoef(pred_matrix.T)
corr_true = np.corrcoef(true_matrix.T)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
KAUST_cmd_cell=CMD_dist(corr_pred,corr_true)

In [625]:
KAUST_cmd_dorc

0.1365082862762953

In [626]:
KAUST_cmd_cell

0.0759634243911973

In [627]:
#MultiVI
true=pd.read_csv("../results/MultiVI/MultiVI_" + train_id + "_" + test_id + "_true.csv",sep=",",index_col=0)
pred=pd.read_csv("../results/MultiVI/MultiVI_" + train_id + "_" + test_id + "_pred.csv",sep=",",index_col=0)

indexes=[pred.index[i].split('_')[0] for i in range(len(pred.index))]
pred.index=indexes
peak_trim=true.columns.values.flatten().tolist()
cell_trim=true.index.values.flatten().tolist()

pred=pred.loc[cell_trim]
idx = np.unique(pred.index, return_index=True)[1]
pred=pred.iloc[idx]
overlap=list(set(pred.columns).intersection(set(true.columns)))
pred=pred.loc[:,overlap]
true=true.loc[:,overlap]
pred=pred.T.sort_index()
true=true.T.sort_index()

In [632]:
overlap=list(set(DORC_gene_matrix.columns.values.flatten()).intersection(set(true.index.values.flatten())))

In [633]:
true_1 = true.loc[overlap,]

In [634]:
pred_1 = pred.loc[overlap,]

In [635]:
DORC_gene_matrix = DORC_gene_matrix.loc[:,overlap]

In [637]:
#得到gene cell矩阵 
true_matrix=DORC_gene_matrix.dot(true_1)

In [638]:
pred_matrix=DORC_gene_matrix.dot(pred_1)

In [639]:
pccs_row=[]
for i in range(0,true_matrix.shape[0]):
    corr = np.corrcoef(true_matrix.iloc[i], pred_matrix.iloc[i])[0,1]
    pccs_row.append(corr)

In [640]:
pccs_col=[]
for i in range(0,true_matrix.shape[1]):
    corr = np.corrcoef(true_matrix.iloc[:,i], pred_matrix.iloc[:,i])[0,1]
    pccs_col.append(corr)

In [641]:
pccs_row_df=pd.DataFrame(data=pccs_row)
np.median(pccs_row_df)

0.41358751133620586

In [642]:
pccs_col_df=pd.DataFrame(data=pccs_col)
np.nanmedian(pccs_col_df)

0.5990529006887665

In [643]:
pccs_row_df.to_csv("../results/DORC/MultiVI_" + train_id + "_" + test_id + "_dorc_pcc.csv")
pccs_col_df.to_csv("../results/DORC/MultiVI_" + train_id + "_" + test_id + "_cell_pcc.csv")

In [644]:
def CMD_dist(A, B):
    x = np.trace(A.dot(B))
    y = np.linalg.norm(A,'fro')*np.linalg.norm(B,'fro')
    return 1- x/(y+1e-8)

In [645]:
corr_pred = np.corrcoef(pred_matrix)
corr_true = np.corrcoef(true_matrix)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
MultiVI_cmd_dorc=CMD_dist(corr_pred,corr_true)

In [646]:
corr_pred = np.corrcoef(pred_matrix.T)
corr_true = np.corrcoef(true_matrix.T)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
MultiVI_cmd_cell=CMD_dist(corr_pred,corr_true)

In [647]:
MultiVI_cmd_dorc

0.2934559632471905

In [648]:
MultiVI_cmd_cell

0.06418920048583887

In [673]:
#BABEL
true=pd.read_csv("../results/BABEL/BABEL_" + train_id + "_" + test_id + "_true.csv",sep=",",index_col=0)
pred=pd.read_csv("../results/BABEL/BABEL_" + train_id + "_" + test_id + "_pred.csv",sep=",",index_col=0)true=true.T
pred.index=true.index
pred.columns=true.columns
pred=pred.sort_index()
true=true.sort_index()
pred=pred.T.sort_index()
true=true.T.sort_index()

In [684]:
pred=pred.T.sort_index()
true=true.T.sort_index()

In [676]:
peaks,index_y =np.unique(DORC.peak,return_inverse=True)
genes,index_x =np.unique(DORC.gene,return_inverse=True)
DORC_gene= coo_matrix((np.ones(shape=(len(index_x),)),(index_x,index_y)))

In [677]:
DORC_gene.index=genes
DORC_gene.columns=peaks

In [678]:
#dorc转数据框
DORC_gene_matrix=pd.DataFrame(DORC_gene.toarray(),index=DORC_gene.index,columns=DORC_gene.columns)

In [679]:
DORC_gene_matrix=DORC_gene_matrix[(DORC_gene_matrix.sum(axis=1)>5)]

In [680]:
DORC_gene_matrix

,chr1-100037674-100038480,chr1-100038644-100039418,chr1-1000558-1001151,chr1-100184356-100185217,chr1-10022422-10023370,chr1-10028788-10029730,chr1-10032403-10033291,chr1-100351075-100351920,chr1-100391470-100392358,chr1-100400115-100401032,...,chrX-79081613-79082535,chrX-79133628-79134549,chrX-79258820-79259745,chrX-79325001-79325941,chrX-79360959-79361907,chrX-79366750-79367634,chrX-79400362-79401306,chrX-81201555-81202477,chrX-9312906-9313844,chrX-96722522-96723463
A2M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAK1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAMP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AATK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ABCA1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZNF860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZNHIT2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZNRF2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [685]:
true_1 = true.loc[DORC_gene_matrix.columns.values.flatten(),]

In [686]:
pred_1 = pred.loc[DORC_gene_matrix.columns.values.flatten(),]

In [687]:
#得到gene cell矩阵 
true_matrix=DORC_gene_matrix.dot(true_1)

In [688]:
pred_matrix=DORC_gene_matrix.dot(pred_1)

In [689]:
pred_matrix=pred_matrix.loc[(pred_matrix!=0).any(axis=1)]

In [690]:
true_matrix=true_matrix.loc[(true_matrix!=0).any(axis=1)]

In [691]:
overlap=list(set(true_matrix.index).intersection(set(pred_matrix.index)))

In [692]:
pred_matrix_1=pred_matrix.loc[overlap,]

In [693]:
true_matrix_1=true_matrix.loc[overlap,]

In [694]:
def CMD_dist(A, B):
    x = np.trace(A.dot(B))
    y = np.linalg.norm(A,'fro')*np.linalg.norm(B,'fro')
    return 1- x/(y+1e-8)

In [695]:
corr_pred = np.corrcoef(pred_matrix_1)
corr_true = np.corrcoef(true_matrix_1)
corr_pred=pd.DataFrame(corr_pred)
corr_true=pd.DataFrame(corr_true)
BABEL_cmd_dorc=CMD_dist(corr_pred,corr_true)

In [696]:
BABEL_cmd_dorc

0.40267689535876905

In [668]:
data_cell = [['LIGER',LIGER_cmd_cell],['Seurat',Seurat_cmd_cell],['CMAE',CMAE_cmd_cell],['BABEL',BABEL_cmd_cell],['DANCE',DANCE_cmd_cell],['KAUST',KAUST_cmd_cell],['MultiVI',MultiVI_cmd_cell]]
df_cell = pd.DataFrame(data_cell,columns=['methods','cmd'])

In [669]:
data_dorc = [['LIGER',LIGER_cmd_dorc],['Seurat',Seurat_cmd_dorc],['CMAE',CMAE_cmd_dorc],['BABEL',BABEL_cmd_dorc],['DANCE',DANCE_cmd_dorc],['KAUST',KAUST_cmd_dorc],['MultiVI',MultiVI_cmd_dorc]]
df_dorc = pd.DataFrame(data_dorc,columns=['methods','cmd'])

In [670]:
df_dorc.to_csv("../results/DORC/" + train_id + "_" + test_id + "_dorc_cmd.csv")
df_cell.to_csv("../results/DORC/" + train_id + "_" + test_id + "_cell_cmd.csv")